In [1]:
import pandas as pd
import numpy as np

'''
Create a sample logging data for a ecommerce website with the following columns:
- user_id
- timestamp
- page_name
- action (view, click)

The data should have at least 1000 rows.
- user_id should be a random number between 1 and 10
- timestamp should be between 2024-01-01 and 2024-12-31
- page_name should be ['home', 'product', 'cart', 'checkout', 'confirmation']
- action should be ['view', 'click']
''' 

        date  store  item  quantity  sale_price
0 2024-11-15      8     3        47         970
1 2024-07-13      5    29        57         550
2 2024-09-21      6    45        60         830
3 2023-07-12      3    12        33         210
4 2024-12-08      7    11        98         530
